In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train=pd.read_csv('/Users/appobs/Desktop/hw/478/week4/hw2/newsgroups/trainMatrixModified.txt',header=None,sep='\t')
tr_lb=pd.read_csv('/Users/appobs/Desktop/hw/478/week4/hw2/newsgroups/trainClasses.txt',header=None,sep='\t')
test=pd.read_csv('/Users/appobs/Desktop/hw/478/week4/hw2/newsgroups/testMatrixModified.txt',header=None,sep='\t')
te_lb=pd.read_csv('/Users/appobs/Desktop/hw/478/week4/hw2/newsgroups/testClasses.txt',header=None,sep='\t')

train.T.head()
test.T.head()

,0,1,2,3,4,5,6,7,8,9,...,5490,5491,5492,5493,5494,5495,5496,5497,5498,5499
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
def knn_classifier(tr_dt,tr_lb,object,k,measure):
    #training and testing datasets are transposed
    tr_dt=tr_dt.T
    tr_dt=np.array(tr_dt)
    object=object.T
    object=np.array(object)
    
    if measure == 0: #euclidean distances
        dists = np.sqrt(((tr_dt - object)**2).sum(axis=1))
    
    elif measure ==1: #cosine similarity
        D_norm = np.array([np.linalg.norm(tr_dt[i]) for i in range(len(tr_dt))])
        x_norm = np.linalg.norm(object)
        sims = np.dot(tr_dt,object)/(D_norm * x_norm)
        dists = 1 - sims
        
    idx = np.argsort(dists)[:k]
    label=[]
    #voting for class
    for i in idx:   
        label.append(tr_lb[1][i])
    if label.count(1)>label.count(0):
        clas =1
    elif label.count(1)<label.count(0):
        clas =0
    elif label.count(1)==label.count(0):
        clas=label[0]
    return clas, sorted(dists)[:k]  

In [4]:
def accuracy_knn(test,te_lb,train,tr_lb,k,measure):
    te_lb=te_lb[1]
    match=0
    for i in range(200):
        tag,array = knn_classifier(train,tr_lb,test[i],k,measure)
        if tag == te_lb[i]:
            match += 1
    accuracy= match/200
    return accuracy

In [5]:
#euclidean distances
for i in range(1,5):
    a=accuracy_knn(test,te_lb,train,tr_lb,i,0)
    print('k= {}: accuracy= {}'.format(i,a))    

k= 1: accuracy= 0.78
k= 2: accuracy= 0.78
k= 3: accuracy= 0.81
k= 4: accuracy= 0.81


In [6]:
#cosine similarity
for i in range(1,5):
    a=accuracy_knn(test,te_lb,train,tr_lb,i,1)
    print('k= {}: accuracy= {}'.format(i,a))    

k= 1: accuracy= 0.985
k= 2: accuracy= 0.985
k= 3: accuracy= 0.97
k= 4: accuracy= 0.985


In [7]:
#### Convert term weights to TFxIDF weights

#Find doucment frequencies for each term
DF = np.array([(train!=0).sum(0)])
NDocs = len(train[0])
# Create a matrix with all entries = NDocs
NMatrix=np.ones(np.shape(train), dtype=float)*NDocs
# Convert each entry into IDF values
# Note that IDF is only a function of the term, so all rows will be identical.
DivM = np.divide(NMatrix, DF)
IDF = np.log2(DivM)
# Finally compute the TFxIDF values for each document-term entry
DT_tfidf = train * IDF

In [8]:
#with TF_IDF converted data
for i in range(1,21):
    b=accuracy_knn(test,te_lb,DT_tfidf,tr_lb,i,0)
    print('k= {}: accuracy= {}'.format(i,b))    

k= 1: accuracy= 0.515
k= 2: accuracy= 0.515
k= 3: accuracy= 0.515
k= 4: accuracy= 0.51
k= 5: accuracy= 0.51
k= 6: accuracy= 0.51
k= 7: accuracy= 0.505
k= 8: accuracy= 0.505
k= 9: accuracy= 0.5
k= 10: accuracy= 0.575
k= 11: accuracy= 0.515
k= 12: accuracy= 0.67
k= 13: accuracy= 0.52
k= 14: accuracy= 0.565
k= 15: accuracy= 0.53
k= 16: accuracy= 0.56
k= 17: accuracy= 0.53
k= 18: accuracy= 0.545
k= 19: accuracy= 0.52
k= 20: accuracy= 0.54


In [9]:
#### Classification for accuracy